### Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

 Задание №1.

Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.


In [1]:
!pip install tensorflow

In [2]:
!pip install keras

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import warnings

warnings.filterwarnings("ignore")

In [4]:
data_reg = pd.read_csv('csgo.csv')
data_cls = pd.read_csv('wines.csv')

In [5]:
data_reg.head(5)

,Unnamed: 0,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,bomb_planted_True
0,20,29.27,2.0,1.0,379.0,99.0,487.0,93.0,2050.0,100.0,0.0,1.0,0.0,5.0,1.0,1
1,26,22.70,3.0,1.0,100.0,149.0,100.0,175.0,1050.0,1250.0,1.0,2.0,1.0,1.0,2.0,1
2,33,37.38,3.0,2.0,300.0,414.0,300.0,441.0,250.0,6300.0,1.0,5.0,0.0,3.0,5.0,1
3,34,17.38,3.0,2.0,300.0,414.0,300.0,441.0,250.0,6300.0,1.0,5.0,0.0,3.0,5.0,1
4,39,24.84,3.0,3.0,67.0,0.0,96.0,0.0,2150.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [6]:
data_reg = data_reg.drop(['Unnamed: 0'], axis=1)
data_reg.head(5)

,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,bomb_planted_True
0,29.27,2.0,1.0,379.0,99.0,487.0,93.0,2050.0,100.0,0.0,1.0,0.0,5.0,1.0,1
1,22.70,3.0,1.0,100.0,149.0,100.0,175.0,1050.0,1250.0,1.0,2.0,1.0,1.0,2.0,1
2,37.38,3.0,2.0,300.0,414.0,300.0,441.0,250.0,6300.0,1.0,5.0,0.0,3.0,5.0,1
3,17.38,3.0,2.0,300.0,414.0,300.0,441.0,250.0,6300.0,1.0,5.0,0.0,3.0,5.0,1
4,24.84,3.0,3.0,67.0,0.0,96.0,0.0,2150.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [7]:
data_cls.head(5)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1,6.3,0.30,0.34,1.6,0.049,14.0,0.9940,3.30,0.49,9.5,6
1,2,8.1,0.28,0.40,6.9,0.050,30.0,0.9951,3.26,0.44,10.1,6
2,3,7.2,0.23,0.32,8.5,0.058,47.0,0.9956,3.19,0.40,9.9,6
3,6,6.2,0.32,0.16,7.0,0.045,30.0,0.9949,3.18,0.47,9.6,6
4,9,8.1,0.22,0.43,1.5,0.044,28.0,0.9938,3.22,0.45,11.0,6


In [8]:
data_cls = data_cls.drop(['Unnamed: 0'], axis=1)
data_cls.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6.3,0.30,0.34,1.6,0.049,14.0,0.9940,3.30,0.49,9.5,6
1,8.1,0.28,0.40,6.9,0.050,30.0,0.9951,3.26,0.44,10.1,6
2,7.2,0.23,0.32,8.5,0.058,47.0,0.9956,3.19,0.40,9.9,6
3,6.2,0.32,0.16,7.0,0.045,30.0,0.9949,3.18,0.47,9.6,6
4,8.1,0.22,0.43,1.5,0.044,28.0,0.9938,3.22,0.45,11.0,6


# Регрессия

In [9]:
y_reg = data_reg['bomb_planted_True']
X_reg = data_reg.drop(['bomb_planted_True'], axis=1)

In [10]:
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=.2)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_reg_train)
X_test_scaled = scaler.transform(X_reg_test)

In [12]:
# создаем модель, как набор последовательных слоев
model_reg = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(14,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Выходной слой без активации для задачи регрессии
])

In [13]:
model_reg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                960       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3073 (12.00 KB)
Trainable params: 3073 (12.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Компиляция модели
model_reg.compile(optimizer='adam', loss='mean_squared_error')

In [17]:
# Обучение модели
model_reg.fit(X_reg_train, y_reg_train, epochs=50, batch_size=32)

Epoch 1/50
662/662 [==============================] - 1s 2ms/step - loss: 0.3936
Epoch 2/50
662/662 [==============================] - 1s 2ms/step - loss: 0.4817
Epoch 3/50
662/662 [==============================] - 1s 1ms/step - loss: 1.9663
Epoch 4/50
662/662 [==============================] - 1s 2ms/step - loss: 0.8852
Epoch 5/50
662/662 [==============================] - 1s 2ms/step - loss: 0.3617
Epoch 6/50
662/662 [==============================] - 1s 2ms/step - loss: 2.1497
Epoch 7/50
662/662 [==============================] - 1s 2ms/step - loss: 0.2813
Epoch 8/50
662/662 [==============================] - 1s 2ms/step - loss: 0.4740
Epoch 9/50
662/662 [==============================] - 1s 1ms/step - loss: 0.2499
Epoch 10/50
662/662 [==============================] - 1s 2ms/step - loss: 0.4054
Epoch 11/50
662/662 [==============================] - 1s 2ms/step - loss: 0.1429
Epoch 12/50
662/662 [==============================] - 1s 2ms/step - loss: 0.1005
Epoch 13/50
662/662 [====

In [18]:
# Оценка модели
y_pred = model_reg.predict(X_reg_test)
mse = mean_squared_error(y_reg_test, y_pred)
r2 = r2_score(y_reg_test, y_pred)
print(f'Mean Squared Error (Regression): {mse}')
print(f'R2 Score (Regression): {r2}')

166/166 [==============================] - 0s 1ms/step
Mean Squared Error (Regression): 0.07335418894798788
R2 Score (Regression): 0.7065647612628019


# Классификация

In [19]:
y_cls = data_cls['quality']
X_cls = data_cls.drop(['quality'], axis=1)

In [20]:
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X_cls, y_cls, test_size=.2)

In [21]:
# Построение модели
model_cls = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_cls_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='softmax')# Выходной слой с функцией активации softmax для классификации
])

In [22]:
# Компиляция модели
model_cls.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Обучение модели
model_cls.fit(X_cls_train, y_cls_train, epochs=50, batch_size=32)

Epoch 1/50

102/102 [==============================] - 1s 2ms/step - loss: 2.3026 - accuracy: 0.3750
Epoch 2/50
102/102 [==============================] - 0s 2ms/step - loss: 1.1352 - accuracy: 0.4657
Epoch 3/50
102/102 [==============================] - 0s 2ms/step - loss: 1.1105 - accuracy: 0.4703
Epoch 4/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0925 - accuracy: 0.4822
Epoch 5/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0836 - accuracy: 0.4786
Epoch 6/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0638 - accuracy: 0.4899
Epoch 7/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0532 - accuracy: 0.4960
Epoch 8/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0473 - accuracy: 0.4979
Epoch 9/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0447 - accuracy: 0.5028
Epoch 10/50
102/102 [==============================] - 0s 2ms/step - loss: 1.0317 - accuracy: 0.517

In [24]:
# Оценка модели
y_pred_prob = model_cls.predict(X_cls_test)
y_pred = tf.argmax(y_pred_prob, axis=1)

accuracy = accuracy_score(y_cls_test, y_pred)
print(f'Accuracy (Classification): {accuracy}')

26/26 [==============================] - 0s 2ms/step
Accuracy (Classification): 0.5532435740514076


# Переход к MLP

Задание №2.

Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.

### Регрессия

In [25]:
class MLP_reg:
    def __init__(self, layer_sizes, activation_functions):
        self.layer_sizes = layer_sizes
        self.activation_functions = activation_functions
        self.weights = []
        self.biases = []
        self.activations = []
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]))
            self.biases.append(np.random.randn(layer_sizes[i + 1]))
            self.activations.append(np.zeros(layer_sizes[i + 1]))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def relu(self, x):
        return np.maximum(0, x)

    def derivative(self, x, activation_function):
        if activation_function == 'sigmoid':
            return x * (1 - x)
        elif activation_function == 'tanh':
            return 1 - np.power(x, 2)
        elif activation_function == 'relu':
            return (x > 0).astype(float)

    def forward(self, x):
        for i in range(len(self.weights)):
            x = np.dot(x, self.weights[i]) + self.biases[i]
            if self.activation_functions[i] == 'sigmoid':
                x = self.sigmoid(x)
            elif self.activation_functions[i] == 'tanh':
                x = self.tanh(x)
            elif self.activation_functions[i] == 'relu':
                x = self.relu(x)
            self.activations[i] = x
        return x

    def backward(self, x, y, learning_rate):
        output = self.forward(x)
        deltas = []
        for i in reversed(range(len(self.weights))):
            if i == len(self.weights) - 1:
                error = output - y
            else:
                error = np.dot(deltas[-1], self.weights[i + 1].T)
            delta = error * self.derivative(self.activations[i], self.activation_functions[i])
            deltas.append(delta)
        deltas.reverse()
        for i in range(len(self.weights)):
            self.weights[i] -= learning_rate * np.dot(x.T if i == 0 else self.activations[i - 1].T, deltas[i])
            self.biases[i] -= learning_rate * np.sum(deltas[i], axis=0)

    def train(self, X, y, learning_rate, epochs):
        for epoch in range(epochs):
            self.backward(X, y, learning_rate)

    def r2_score(self, y_true, y_pred):
        ss_res = np.sum(np.square(y_true - y_pred))
        ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
        return 1 - ss_res / ss_tot

In [26]:
data_reg2 = pd.read_csv("wines.csv")

def preprocess_data(trip_data):
    X = data_reg2.drop(columns=['quality']).to_numpy()
    y = data_reg2['quality'].to_numpy().reshape(-1, 1)
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    y = (y - y.mean()) / y.std()
    return X, y

# Подготовка данных
X, y = preprocess_data(data_reg2)

# Инициализация и обучение модели
layer_sizes = [X.shape[1], 10, 5, 1]  # Примерная архитектура сети
activation_functions = ['tanh', 'relu', 'tanh']
mlp = MLP_reg(layer_sizes, activation_functions)
mlp.train(X, y, learning_rate=0.01, epochs=100)

# Проверка модели
y_pred = mlp.forward(X)
r2 = mlp.r2_score(y, y_pred)
print(f"r2_score: {r2}")

r2_score: 0.03952215953605731


### Классификация

In [27]:
class MLP_cls:
    def __init__(self, layer_sizes, activation_functions):
        self.layer_sizes = layer_sizes
        self.activation_functions = activation_functions
        self.weights = [np.random.randn(y, x) for x, y in zip(layer_sizes[:-1], layer_sizes[1:])]
        self.biases = [np.random.randn(y, 1) for y in layer_sizes[1:]]

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def tanh(self, z):
        return np.tanh(z)

    def tanh_derivative(self, z):
        return 1 - np.tanh(z) ** 2

    def relu(self, z):
        return np.maximum(0, z)

    def relu_derivative(self, z):
        return (z > 0) * 1

    def forward_propagation(self, input_data):

        for b, w, activation in zip(self.biases, self.weights, self.activation_functions):
            if activation == 'sigmoid':
                input_data = self.sigmoid(np.dot(w, input_data) + b)
            elif activation == 'tanh':
                input_data = self.tanh(np.dot(w, input_data) + b)
            elif activation == 'relu':
                input_data = self.relu(np.dot(w, input_data) + b)
        return input_data

    def backpropagation(self, input_data, target):

        gradient_b = [np.zeros(b.shape) for b in self.biases]
        gradient_w = [np.zeros(w.shape) for w in self.weights]


        activation = input_data
        activations = [input_data]
        zs = []
        for b, w, activation_function in zip(self.biases, self.weights, self.activation_functions):
            z = np.dot(w, activation) + b
            zs.append(z)
            if activation_function == 'sigmoid':
                activation = self.sigmoid(z)
            elif activation_function == 'tanh':
                activation = self.tanh(z)
            elif activation_function == 'relu':
                activation = self.relu(z)
            activations.append(activation)

        # Backward pass
        delta = self.cost_derivative(activations[-1], target) * self.get_activation_derivative(activations[-1], self.activation_functions[-1])
        gradient_b[-1] = delta
        gradient_w[-1] = np.dot(delta, activations[-2].T)

        for l in range(2, len(self.layer_sizes)):
            z = zs[-l]
            activation_derivative = self.get_activation_derivative(z, self.activation_functions[-l])
            delta = np.dot(self.weights[-l + 1].T, delta) * activation_derivative
            gradient_b[-l] = delta
            gradient_w[-l] = np.dot(delta, activations[-l - 1].T)

        return gradient_b, gradient_w

    def get_activation_derivative(self, z, activation_function):

        if activation_function == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif activation_function == 'tanh':
            return self.tanh_derivative(z)
        elif activation_function == 'relu':
            return self.relu_derivative(z)
    def cost_derivative(self, output_activations, target):

        return output_activations - target

    def update_parameters(self, mini_batch, learning_rate):

        sum_gradient_b = [np.zeros(b.shape) for b in self.biases]
        sum_gradient_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_gradient_b, delta_gradient_w = self.backpropagation(x, y)
            sum_gradient_b = [nb + dnb for nb, dnb in zip(sum_gradient_b, delta_gradient_b)]
            sum_gradient_w = [nw + dnw for nw, dnw in zip(sum_gradient_w, delta_gradient_w)]

        self.weights = [w - (learning_rate / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, sum_gradient_w)]
        self.biases = [b - (learning_rate / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, sum_gradient_b)]

In [28]:
X = data_reg.drop(['bomb_planted_True'], axis=1)
y = data_reg['bomb_planted_True'].values.reshape(-1, 1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Подготовка данных для MLP
X_train = [X_train[i].reshape(-1, 1) for i in range(X_train.shape[0])]
y_train = [y_train[i].reshape(-1, 1) for i in range(y_train.shape[0])]
training_data = list(zip(X_train, y_train))

# Определение параметров сети
input_size = X.shape[1]
hidden_layers = [10, 5]
output_size = 1

# Создание и обучение MLP
mlp = MLP_cls([input_size] + hidden_layers + [output_size], ['relu'] * len(hidden_layers) + ['sigmoid'])
learning_rate = 0.01
for epoch in range(20):
    np.random.shuffle(training_data)
    mini_batches = [training_data[k:k+10] for k in range(0, len(training_data), 10)]
    for mini_batch in mini_batches:
        mlp.update_parameters(mini_batch, learning_rate)

def predict(mlp, X):
    predictions = [mlp.forward_propagation(x.reshape(-1, 1)) for x in X]
    return np.array(predictions).squeeze()

predictions = predict(mlp, X_test)
predictions_binary = (predictions > 0.5).astype(int)
accuracy = accuracy_score(y_test, predictions_binary)

print("Accuracy:", accuracy)

Accuracy: 0.9168451556003527
